In [1]:
# https://stackoverflow.com/questions/54395735/how-to-work-with-heic-image-file-types-in-python

In [ ]:
# some really cool iPython magic (https://jupyterlite.readthedocs.io/en/latest/howto/pyodide/packages.html#installing-packages-at-runtime)

In [2]:
%pip install -q ipyleaflet

In [3]:
import tomllib

In [4]:
from PIL import Image
from PIL.ExifTags import TAGS, GPSTAGS
from pillow_heif import register_heif_opener

register_heif_opener()

In [6]:
from ipyleaflet import Map, Heatmap, basemaps, basemap_to_tiles, Marker

In [7]:
# from zipfile import ZipFile

In [8]:
img = Image.open('IMG_9398.heic')

In [9]:
# https://stackoverflow.com/questions/72522522/how-to-extract-gps-location-from-heic-files

In [10]:
exif = img.getexif()

In [11]:
# the get_ifd here is getting the image file directory for GPS within the exif
# 0x8825 is the hexademical representation of the GPS fid pointer
ifd = exif.get_ifd(0x8825)

In [12]:
# this is a more verbose option, which you have to provide the GPS labels ahead of time
def get_geotagging(exif):
    geo_tagging_info = {}
    if not exif:
        raise ValueError("No EXIF metadata found")
    else:
        gps_keys = ['GPSVersionID', 'GPSLatitudeRef', 'GPSLatitude', 'GPSLongitudeRef', 'GPSLongitude',
                    'GPSAltitudeRef', 'GPSAltitude', 'GPSTimeStamp', 'GPSSatellites', 'GPSStatus', 'GPSMeasureMode',
                    'GPSDOP', 'GPSSpeedRef', 'GPSSpeed', 'GPSTrackRef', 'GPSTrack', 'GPSImgDirectionRef',
                    'GPSImgDirection', 'GPSMapDatum', 'GPSDestLatitudeRef', 'GPSDestLatitude', 'GPSDestLongitudeRef',
                    'GPSDestLongitude', 'GPSDestBearingRef', 'GPSDestBearing', 'GPSDestDistanceRef', 'GPSDestDistance',
                    'GPSProcessingMethod', 'GPSAreaInformation', 'GPSDateStamp', 'GPSDifferential']

        for k, v in exif.items():
            try:
                geo_tagging_info[gps_keys[k]] = str(v)
            except IndexError:
                pass
        return geo_tagging_info

In [13]:
get_geotagging(ifd)

{'GPSLatitudeRef': 'N',
 'GPSLatitude': '(40.0, 41.0, 48.91)',
 'GPSLongitudeRef': 'W',
 'GPSLongitude': '(73.0, 55.0, 6.09)',
 'GPSAltitudeRef': "b'\\x00'",
 'GPSAltitude': '10.550080475424043',
 'GPSSpeedRef': 'K',
 'GPSSpeed': '0.48419392121950455',
 'GPSImgDirectionRef': 'T',
 'GPSImgDirection': '323.4054870375031',
 'GPSDestBearingRef': 'T',
 'GPSDestBearing': '323.4054870375031',
 'GPSDateStamp': '2021:12:16'}

In [14]:
# this is a much more succinct version where the labels are surfaced using the GPSTAGS
def get_geo(exif):
    for key, value in TAGS.items():
        if value == "GPSInfo":
            break
    gps_info = exif.get_ifd(key)
    return {
        GPSTAGS.get(key, key): value
        for key, value in gps_info.items()
    }

In [15]:
gps = get_geo(exif)

In [16]:
# https://sylvaindurand.org/gps-data-from-photos-with-python/

In [17]:
def get_gps_in_degrees(gps):
    # reading from toml, currently written as array of str
    # coord stored in deg, min, second
    lat = list(map(float, gps['GPSLatitude']))
    lat_ref = gps['GPSLatitudeRef']
    lat = float(lat[0]+(lat[1]/60)+(lat[2]/3600))
    lat = -lat if lat_ref == 'S' else lat

    long = list(map(float, gps['GPSLongitude']))
    long_ref = gps['GPSLongitudeRef']
    long = float(long[0]+(long[1]/60)+(long[2]/3600))
    long = -long if long_ref == 'W' else long

    
    return (lat,long)

In [18]:
# Lat, Long for some cities
nyc_gps = (40.7128, -74.0060)
brooklyn_gps = (40.6782, -73.9442)
bushwick_gps = (40.6958, -73.9171)
paris_gps = (48.8566, 2.3522)

## Photo Locations

In [19]:
m = Map(
    basemap=basemap_to_tiles(basemaps.OpenStreetMap.Mapnik),
    center=bushwick_gps,
    zoom=13
    )

m.add_layer(Marker(location=(get_gps_in_degrees(gps))))
m

Map(center=[40.6958, -73.9171], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'z…

In [20]:
coords = tomllib.load(open('roadent_kill_gps_coords.toml', 'rb'))

In [21]:
coord = list(coords.items())[0][1]['GPSLatitude']

In [22]:
for img_title, coord in coords.items():
    lat_long = get_gps_in_degrees(coord)
    m.add_layer(Marker(location=(lat_long), title=f"{img_title}: {lat_long}"))

## Heatmap

In [23]:
h = Map(
    basemap=basemap_to_tiles(basemaps.OpenStreetMap.Mapnik),
    center=bushwick_gps,
    zoom=13
    )

h.add_layer(Marker(location=(get_gps_in_degrees(gps))))

In [24]:
heatmap_coords = list()

for img_title, coord in coords.items():
     heatmap_coords.append(get_gps_in_degrees(coord))
heatmap = Heatmap(locations=heatmap_coords)
h.add(heatmap)

Map(center=[40.6958, -73.9171], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'z…